In [16]:
import cv2
import mediapipe as mp
import numpy as np
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        
    def __str__(self):
        return f'({self.x}, {self.y})'
    
    def __repr__(self):
        return self.__str__()

def get_user_height_in_pixels(results, frame):

    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        left_foot = landmarks[mp_pose.PoseLandmark.LEFT_HEEL]
        right_foot = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL]
        left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER]
        
        # Calculate the height in pixels from the shoulders to the feet
        shoulder_y = (left_shoulder.y + right_shoulder.y) / 2 * frame.shape[0]
        foot_y = (left_foot.y + right_foot.y) / 2 * frame.shape[0]
        
        height_pixels = abs(foot_y - shoulder_y)
        return height_pixels
    else:
        return None
    
def get_user_position(results, frame):

    
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        left_foot = landmarks[mp_pose.PoseLandmark.LEFT_HEEL]
        right_foot = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL]
        
        # Calculate the midpoint between the left and right feet
        foot_x = (left_foot.x + right_foot.x) / 2 * frame.shape[1]
        foot_y = (left_foot.y + right_foot.y) / 2 * frame.shape[0]
        
        return Point(foot_x, foot_y)
    else:
        return None

def get_total_distance(curr, prev, total_dist):
    if curr and prev:
        # Calculate the distance moved in this frame
        distance_moved = np.linalg.norm(np.array([curr.x, curr.y]) - np.array([prev.x, prev.y]))
        total_dist += distance_moved
        return total_dist
    else:
        return total_dist


In [17]:

def process_gait_speed_walk_test(height, video_path, display = True):
    cap = cv2.VideoCapture(video_path)
    ret, frame = cap.read()
    if not ret:
        logging.error("Failed to read the video")
        cap.release()
        exit()
        
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(image_rgb)
    

    # Get the user's height in pixels from the first frame
    user_height_pixels = get_user_height_in_pixels(results, frame)
    if user_height_pixels is None:
        logging.error("Failed to detect the user in the first frame")
        cap.release()
        exit()

    logging.info(f"User height in pixels: {user_height_pixels}")
    # Calculate the conversion factor from cm to pixels
    cm_to_pixels = user_height_pixels / user_height_cm

    # Calculate the pixel distance equivalent to 4 meters (400 cm)
    distance_4m_pixels = 400 * cm_to_pixels
    logging.info(f"Distance equivalent to 4 meters in pixels: {distance_4m_pixels}")

    # Track the user's movement
    total_distance_walked = 0
    previous_position = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Get the user's current position
        current_position = get_user_position(results, frame)
        total_distance_walked = get_total_distance(current_position, previous_position, total_distance_walked)
        
    
        previous_position = current_position

        # Check if the user has walked the equivalent of 4 meters
        if total_distance_walked >= distance_4m_pixels:
            logging.info("The user has walked 4 meters. Stopping the test.")
            break

    cap.release()

    logging.info(f"Total distance walked in pixels: {total_distance_walked}")


In [18]:
video_path = '/Users/brennanlee/Desktop/opencv-healthcare/test/GSWT2.mp4'

user_height_cm = 170  # Example: 170 cm

process_gait_speed_walk_test(user_height_cm, video_path)


2024-07-22 19:58:42,833 - INFO - User height in pixels: 218.43735337257385
2024-07-22 19:58:42,834 - INFO - Distance equivalent to 4 meters in pixels: 513.9702432295855
2024-07-22 19:58:43,075 - INFO - Total distance walked in pixels: 0.0
